<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## 1. Web Scrape and Explore Dataset

In [30]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Scrap list of postal codes in wiki page by using package 'BeautifulSoup'

Convert the content of HTML as dataframe

In [31]:
html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source_page = requests.get(html).text
soup = BeautifulSoup(source_page, 'lxml')
#print(soup.prettify())

In [32]:
#create a dataframe 
column_name = ['Postalcode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_name)

#find postalcode, borough and neighborhood
match = soup.find('div', class_='mw-parser-output')
data = match.table.tbody

#loop to extract the data
for tr in data.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postalcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2:
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'Postalcode':postalcode,'Borough':borough,
'Neighborhood':neighborhood}, ignore_index = True)

In [35]:
df = df[df.Borough != 'Not assigned']
df = df[df.Borough != 0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0, df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] == df.iloc[i][1]
        i = i + 1
Toronto = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
Toronto.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [37]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
data1 = Toronto.groupby(['Postalcode', 'Borough'])
toronto = data1.apply(neighborhood_list).reset_index(name='Neighborhood')

In [38]:
print(toronto.shape)
toronto.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
